# Scenario Context
Scenario contexts enable the user to price and calculate risk under varying market states and pricing environments.

In [ ]:
from gs_quant.instrument import IRSwaption
from gs_quant.risk import MarketDataPattern, MarketDataShock, MarketDataShockType, MarketDataShockBasedScenario, \
    CarryScenario, CurveScenario
from gs_quant.session import Environment, GsSession

In [ ]:
# external users should substitute their client id and secret; please skip this step if using internal jupyterhub
GsSession.use(Environment.PROD, client_id=None, client_secret=None, scopes=('run_analytics',))

In [ ]:
# Create and price a swaption
swaption = IRSwaption('Pay', '5y', 'USD', expirationDate='3m')
base_price = swaption.price()
base_price

### MarketDataShockBasedScenario
Allows the user to create a bespoke market data shock

In [ ]:
# Scenario Creation: Shock all points on the Vol Curve by 1bp
ir_vol_scenario = MarketDataShockBasedScenario(
    shocks={
        MarketDataPattern('IR Vol'): MarketDataShock(MarketDataShockType.Absolute, 1e-4)
    }
)

In [ ]:
# Price swaption under scenario
with ir_vol_scenario:
    scenario_price = swaption.price()

scenario_price

In [ ]:
# Swaption Price Comparison: Base vs. Shocked Vol Curves
diff = scenario_price - base_price
diff

In [ ]:
# Comparing Parallel Bump Scenario w/ Vega
from gs_quant.risk import IRVegaParallel
vega = swaption.calc(IRVegaParallel)
vega

### Curve Scenario 
A predefined scenario used to modify the shape of the curve with bespoke transformations 

In [ ]:
# Scenario Creation: modify the Vol Curve by a 5bp parallel shift, 1bp slope shift pivoted at 5y point (up to 50y)
curve_scenario = CurveScenario(MarketDataPattern('IR', 'USD'), annualised_parallel_shift=5, 
                               annualised_slope_shift=1, pivot_point=5, cutoff=50)

with curve_scenario:
    swaption_scenario_price = swaption.price()

# Look at the difference between scenario and base prices
print('Base price:     {:,.2f}'.format(base_price))
print('Scenario price: {:,.2f}'.format(swaption_scenario_price))

### Carry Scenario 
A predefined scenario used to evolve market data and trades over a period of time

In [ ]:
# Carry Scenario - Shift forward 1 month
base_price = swaption.price()
with CarryScenario(date='1m'):
    fwd_price = swaption.price()

print('Base price:     {:,.2f}'.format(base_price))
print('Scenario price: {:,.2f}'.format(fwd_price))
print('Diff: {:,.2f}'.format(fwd_price - base_price))